In [14]:
import numpy as np
import os
import glob
import cv2
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [2]:
def load_dataset(data_dir):
    """
    Load all grayscale .jpg images from a directory and flatten them into column vectors.

    Args:
        data_dir (str): Path to the folder containing .jpg images.

    Returns:
        X (np.ndarray): 2D array of shape (D, N), where D = H * W (flattened image size)
                        and N = number of images.
        labels (List[int]): List of length N with integer labels parsed from the
                            first two characters of each filename.
    """
    # Find and sort all .jpg files in the directory
    paths = sorted(glob.glob(os.path.join(data_dir, '*.jpg')))
    data, labels = [], []

    for p in paths:
        # Read image as grayscale
        img = cv2.imread(p, cv2.IMREAD_GRAYSCALE)
        if img is None:
            # Skip files that cannot be read
            continue

        # Transpose so that flattening matches D = H*W ordering,
        # then convert to float32 and flatten to a vector
        vec = img.T.flatten().astype(np.float32)
        data.append(vec)

        # Extract label from first two characters of the filename
        labels.append(int(os.path.basename(p)[:2]))

    # Stack all vectors into a (D, N) array
    X = np.stack(data, axis=1)
    return X, labels

def compute_pairwise_distances(X, Y, metric='l2'):
    """
    Compute pairwise distances between two sets of vectors using the specified metric.

    Args:
        X (np.ndarray): Array of shape (D, M), representing M query vectors of dimensionality D.
        Y (np.ndarray): Array of shape (D, N), representing N reference vectors of dimensionality D.
        metric (str): Distance metric to use; either 'l2' for Euclidean distance or 'l1' for Manhattan distance.

    Returns:
        np.ndarray: Distance matrix of shape (M, N), where entry (i, j) is the distance between X[:, i] and Y[:, j].

    Raises:
        ValueError: If an unsupported metric string is provided.
    """
    if metric == 'l2':
        # Use Euclidean (L2) distance
        return calc_l2(X, Y)
    elif metric == 'l1':
        # Use Manhattan (L1) distance
        return calc_l1(X, Y)
    else:
        # Reject unknown metrics
        raise ValueError(f"Unknown metric: {metric}")

def calc_l2(X, Y):
    """
    Compute pairwise L2 (Euclidean) distances between two sets of vectors.

    Args:
        X (np.ndarray): Array of shape (D, M), M query vectors.
        Y (np.ndarray): Array of shape (D, N), N reference vectors.

    Returns:
        np.ndarray: Matrix of shape (M, N) where entry (i, j) is
                    ||X[:, i] - Y[:, j]||_2.
    """
    # Expand dims to broadcast subtraction: result shape (D, M, N)
    diff = X[:, :, None] - Y[:, None, :]
    # Sum squared differences over D, then take square root
    return np.sqrt((diff ** 2).sum(axis=0))


def calc_l1(X, Y):
    """
    Compute pairwise L1 (Manhattan) distances between two sets of vectors.

    Args:
        X (np.ndarray): Array of shape (D, M), M query vectors.
        Y (np.ndarray): Array of shape (D, N), N reference vectors.

    Returns:
        np.ndarray: Matrix of shape (M, N) where entry (i, j) is
                    ||X[:, i] - Y[:, j]||_1.
    """
    # Expand dims and sum absolute differences over D
    return np.abs(X[:, :, None] - Y[:, None, :]).sum(axis=0)

def match_accuracy(X_train, y_train, X_test, y_test, metric='l2'):
    """
    Perform nearest-neighbor classification and compute accuracy.

    For each test vector, find the closest training vector using the specified
    distance metric, then compare its label to the true test label.

    Args:
        X_train (np.ndarray): Training data of shape (D, N_train).
        y_train (List[int]): Labels for training data, length N_train.
        X_test (np.ndarray): Test data of shape (D, N_test).
        y_test (List[int]): True labels for test data, length N_test.
        metric (str): Distance metric to use; either 'l1' or 'l2'.

    Returns:
        float: Classification accuracy (between 0 and 1).
    """
    # Choose the appropriate distance function
    if metric == 'l2':
        distances = calc_l2(X_test, X_train)
    elif metric == 'l1':
        distances = calc_l1(X_test, X_train)
    else:
        raise ValueError("metric must be 'l1' or 'l2'")

    # Predict the label of the nearest neighbor for each test sample
    preds = [y_train[np.argmin(dist_row)] for dist_row in distances]

    # Compute mean accuracy
    return np.mean([pred == true for pred, true in zip(preds, y_test)])

In [16]:
def compute_centered_matrix(X_tr):
    """
    Center the input data by subtracting its mean vector.

    Args:
        X_tr (ndarray of shape (D, N)):
            Original data matrix, where D is the feature dimension
            and N is the number of samples.

    Returns:
        X_centered (ndarray of shape (D, N)):
            Data matrix after subtracting the mean from each column.
        mu (ndarray of shape (D,)):
            Mean vector computed across all samples.
    """
    # Compute the mean of each feature (row) over all samples (columns)
    # Subtract the mean from each column to center the data
    mu = np.mean(X_tr, axis=1)
    X_centered = X_tr - mu[:,None]
    return X_centered, mu

In [4]:
def compute_covariance_matrix(X_centered):
    """
    Compute the covariance matrix of centered data.

    Args:
        X_centered (ndarray of shape (D, N)):
            Centered data matrix.

    Returns:
        C (ndarray of shape (D, D)):
            Covariance matrix, where C[i, j] is the covariance
            between feature i and feature j.
    """
    # bias=True uses denominator N instead of N-1
    C = np.cov(X_centered, bias=True)
    return C

In [5]:
def eigen_decomposition(C):
    """
    Perform standard eigen decomposition on a covariance matrix.

    Args:
        C (ndarray of shape (D, D)):
            Covariance matrix.

    Returns:
        eigvals (ndarray of length D):
            Eigenvalues sorted in descending order.
        eigvecs (ndarray of shape (D, D)):
            Corresponding eigenvectors as columns.
    """
    # Compute all eigenvalues and eigenvectors
    # Sort in descending order
    eigvals, eigvecs = np.linalg.eigh(C)
    idx = np.argsort(eigvals)[::-1]
    eigvals = eigvals[idx]
    eigvecs = eigvecs[:, idx]
    return eigvals, eigvecs

In [7]:
def svd_eigen(X_centered):
    """
    Extract singular values from the centered data via SVD.

    Args:
        X_centered (ndarray of shape (D, N)):
            Centered data matrix.

    Returns:
        S (ndarray of length min(D, N)):
            Singular values of X_centered.
    """
    # U and Vt are discarded here; we only need S
    _, S, _ = np.linalg.svd(X_centered, full_matrices=False)
    return S


In [8]:
def svd_eigen_decomposition(S, N):
    """
    Convert singular values to covariance eigenvalues.

    Args:
        S (array-like of length min(D, N)):
            Singular values from SVD.
        N (int):
            Number of samples (used to normalize).

    Returns:
        eigvals_svd (ndarray of length min(D, N)):
            Eigenvalues of the covariance matrix computed as S**2 / N.
    """
    eigvals_svd = (S ** 2) / N
    return eigvals_svd

In [9]:
def projection(X, basis, mu):
    """
    Project data onto a lower-dimensional basis.

    Args:
        X (ndarray of shape (D, N)):
            Original data matrix.
        basis (ndarray of shape (D, k)):
            Matrix whose columns are the top-k basis vectors.
        mu (ndarray of shape (D,)):
            Mean vector used for centering.

    Returns:
        P (ndarray of shape (k, N)):
            Coordinates of each sample in the new basis.
    """
    # Center X and then multiply by the basis vectors
    X_centered = X - mu[:, None]
    P = basis.T @ X_centered
    return P

In [10]:
def select_components(eigvals, threshold):
    """
    Determine how many principal components to keep to retain given energy.

    Args:
        eigvals (array-like of length D):
            Eigenvalues sorted in descending order.
        threshold (float in (0, 1]):
            Fraction of total variance to retain.

    Returns:
        k (int):
            Minimum number of leading components such that
            cumulative variance ≥ threshold.
    """
    # searchsorted returns the first index where cum_ratio >= threshold
    cum_ratio = np.cumsum(eigvals) / np.sum(eigvals)
    k = np.searchsorted(cum_ratio, threshold) + 1
    return k

In [11]:
def proj2(data_root):
    # Load training and testing data
    X_tr, y_tr = load_dataset(os.path.join(data_root, 'train'))
    X_te, y_te = load_dataset(os.path.join(data_root, 'test'))

    # Determine dimensions
    D, N = X_tr.shape

    # Part 1: Covariance-based Eigen Decomposition
    print("=== Part 1: Covariance Eigen Decomposition ===")
    X_centered, mu = compute_centered_matrix(X_tr)
    C = compute_covariance_matrix(X_centered)
    eigvals, eigvecs = eigen_decomposition(C)
    print(f"Computed {D} eigenvalues from {D}×{D} covariance matrix.")
    print(f"Top-10 eigenvalues: {eigvals[:10]}\n")

    # Part 2: SVD Eigen Decomposition
    print("=== Part 2: SVD Eigen Decomposition ===")
    S = svd_eigen(X_centered)
    eigvals_svd = svd_eigen_decomposition(S, N)
    print(f"Computed {len(S)} singular values from {D}×{N} centered data.")
    print(f"Top-10 SVD-based eigenvalues: {eigvals_svd[:10]}")

    # Optional similarity check using L1 / L2 distances
    padded_svd = np.concatenate([eigvals_svd, np.zeros(D - len(eigvals_svd))])
    X_vec = eigvals[:, None]  # shape (D, 1)
    Y_vec = padded_svd[:, None]  # shape (D, 1)

    # compute L1 and L2 distances
    l1_dist = compute_pairwise_distances(X_vec, Y_vec, metric='l1')[0, 0]
    l2_dist = compute_pairwise_distances(X_vec, Y_vec, metric='l2')[0, 0]
    print(f"Eigenvalue sequences L1 distance: {l1_dist:.6f}")
    print(f"Eigenvalue sequences L2 distance: {l2_dist:.6f}\n")


    # Part 3: Projection and Matching Accuracy
    print("=== Part 3: Projection & Matching Accuracy ===")
    def match_on_basis(basis, label):
        P_tr = projection(X_tr, basis, mu)
        P_te = projection(X_te, basis, mu)
        acc = match_accuracy(P_tr, y_tr, P_te, y_te, 'l2')
        print(f"{label}: L2 accuracy = {acc*100:.2f}%")

    match_on_basis(eigvecs, 'Full basis')

    # Part 4: Energy Threshold Impact
    print("\n=== Part 4: Energy Threshold Impact ===")
    for thr in (0.90, 0.95):
        k = select_components(eigvals, thr)
        basis_k = eigvecs[:, :k]
        acc_k = match_accuracy(projection(X_tr, basis_k, mu), y_tr,
                               projection(X_te, basis_k, mu), y_te, 'l2')
        storage_drop = (1 - k/D) * 100
        print(f"Energy {int(thr*100)}% (k={k}): accuracy={acc_k*100:.2f}%, storage↓{storage_drop:.1f}%")


In [18]:
data_root = '/content/drive/MyDrive/HW1/data/'  # folders: data/train, data/test, data/test2
print("\n===== Project 2 =====")
proj2(data_root)



===== Project 2 =====
=== Part 1: Covariance Eigen Decomposition ===
Computed 1764 eigenvalues from 1764×1764 covariance matrix.
Top-10 eigenvalues: [2738657.29557341  937279.85673454  331728.34354099  186819.89489465
  141655.12260084  114923.07160831  108920.02556153   85017.9533288
   67741.6417524    58794.04244069]

=== Part 2: SVD Eigen Decomposition ===
Computed 350 singular values from 1764×350 centered data.
Top-10 SVD-based eigenvalues: [2738657.2   937279.8   331728.34  186819.89  141655.12  114923.08
  108920.02   85017.95   67741.65   58794.04]
Eigenvalue sequences L1 distance: 0.174070
Eigenvalue sequences L2 distance: 0.065031

=== Part 3: Projection & Matching Accuracy ===
Full basis: L2 accuracy = 96.00%

=== Part 4: Energy Threshold Impact ===
Energy 90% (k=37): accuracy=96.00%, storage↓97.9%
Energy 95% (k=81): accuracy=94.00%, storage↓95.4%
